# H₂ Molecule Homework Assignment
### Quantum Software Development Journey: From Theory to Application with Classiq - Part 3

- Similarly to what we have done in class, in this exercise we will implement the VQE on H2 molecule.
- This time instead of using the built-in methods and functions (such as `Molecule` and `MoleculeProblem`) to difne and solve the problem, you will be provided with a two qubits Hamiltonian.

## Submission
- Submit the completed Jupyter notebook and report via GitHub. Ensure all files are correctly named and organized.
- Use the Typeform link provided in the submission folder to confirm your submission.

## Additional Resources
- [Classiq Documentation](https://docs.classiq.io/latest/)
- The notebook from live session #3

## Important Dates
- **Assignment Release:** 22.5.2024
- **Submission Deadline:** 3.6.2024 (7 A.M GMT+3)

---

Happy coding and good luck!

### Part 1

Given the following Hamiltonian:

$$
\hat{H} = -1.0523 \cdot (I \otimes I) + 0.3979 \cdot (I \otimes Z) - 0.3979 \cdot (Z \otimes I) - 0.0112 \cdot (Z \otimes Z) + 0.1809 \cdot (X \otimes X)
$$

Complete the following code

In [1]:
from typing import List
from classiq import *

HAMILTONIAN = QConstant("HAMILTONIAN", List[PauliTerm],
                        [PauliTerm(pauli=[Pauli.I, Pauli.I], coefficient=-1.0523),
                         PauliTerm(pauli=[Pauli.I, Pauli.Z], coefficient=0.3979),
                         PauliTerm(pauli=[Pauli.Z, Pauli.I], coefficient=-0.3979),
                         PauliTerm(pauli=[Pauli.Z, Pauli.Z], coefficient=-0.0112),
                         PauliTerm(pauli=[Pauli.X, Pauli.X], coefficient=0.1809)]) #TODO: Complete Hamiltonian

In [2]:
@qfunc
def main(q: Output[QArray[QBit]], angles: CArray[CReal, 3]) -> None:
    # TODO: Create an ansatz which allows each qubit to have arbitrary rotation
    allocate(2, q)
    U(angles[0], angles[1], angles[2], 0, q[0])
    U(angles[0], angles[1], angles[2], 0, q[1])


@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN, # TODO: complete the missing argument
        False,
        [],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

qmod = create_model(main, classical_execution_function=cmain) #TODO: complete the line, use classical_execution_function
qprog = synthesize(qmod)
# show(qprog)

The current version of 'classiq' has been deprecated, and will not be supported as of 2024-06-27. Please run "pip install -U classiq" to upgrade the classiq SDK to the latest version.


In [3]:
execution = execute(qprog)
res = execution.result()
# execution.open_in_ide()
vqe_result = res[0].value #TODO: complete the line

In [4]:
print(f"Optimal energy: {vqe_result.energy}")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.0646799804687501
Optimal parameters: {'angles_0': -6.164185814953359, 'angles_1': -5.172941087398459, 'angles_2': -5.722759806197389}
Eigenstate: {'01': (0.05846339666834283+0j), '10': (0.05412658773652741+0j), '00': (0.9968211192335363+0j)}


In [5]:
# Optimal parameters: {'angles_0': -3.0292653800801213, 'angles_1': 0.13084292242999915, 'angles_2': -5.614248543217801}
# Eigenstate: {'10': (0.07328774624724109+0j), '11': (0.07328774624724109+0j), '01': (0.9946144039274718+0j)}

The optimal energy is different from what was calculated in class.

Does it similar to the `optimal energy` we calculated in class? \
Does it similar to the `total energy` we calculated in class?

### Part 2

**Now, we want to have a more interesting ansatz in our `main`.**  
Add **one** line of code to the `main` function you created in Part 1 that creates **entanglement** between the two qubits.  
Which gate should you use?

In [6]:
@qfunc
def main(q: Output[QArray[QBit]], angles: CArray[CReal, 3]) -> None:
    # TODO: Create an ansatz which allows each qubit to have arbitrary rotation
    allocate(2, q)
    U(angles[0], angles[1], angles[2], 0, q[0])
    U(angles[0], angles[1], angles[2], 0, q[1])
    CX(q[0], q[1])


@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN, # TODO: complete the missing argument
        False,
        [],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

qmod = create_model(main, classical_execution_function=cmain) #TODO: complete the line, use classical_execution_function
qprog = synthesize(qmod)
# show(qprog)

In [7]:
execution = execute(qprog)
res = execution.result()
# execution.open_in_ide()
vqe_result = res[0].value #TODO: complete the line

In [8]:
print(f"Optimal energy: {vqe_result.energy}")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.8396752929687499
Optimal parameters: {'angles_0': -3.3237005318854607, 'angles_1': 3.9061632465634935, 'angles_2': -0.840172938028935}
Eigenstate: {'11': (0.07328774624724109+0j), '00': (0.02209708691207961+0j), '10': (0.09375+0j), '01': (0.9926487608917869+0j)}


The optimal energy is close to what was calculated in class.

Does it similar to the `optimal energy` we calculated in class? \
Does it similar to the `total energy` we calculated in class? \
What can we learn about the provided  form this result Hamiltonian?